In [5]:
%%writefile localSGD.py

import os
import math
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import argparse
from torch.utils.data import random_split, Subset, DataLoader
from google.colab import files

# Define hyperparameters
# K_values = [2, 4, 8]
# J_values = [4, 8, 16, 32, 64]
# Alpha slow learning rate a=1 best
# Beta slow momentum
global_iteration_per_epoch = 782 # to be divided by K*J
num_epochs = 150
batch_size = 64
base_learning_rate = 1e-3
momentum = 0.9
weight_decay = 4e-4
#warmup_epochs = 5

alpha = 1.0  # Slow learning rate scaling factor
beta = 0.5  # Slow momentum factor

def compute_mean_std(dataset):
    """Compute the mean and std of CIFAR-100 dataset.

    Args:
        dataset: A dataset derived from `torch.utils.data.Dataset`,
                 such as `cifar100_training_dataset` or `cifar100_test_dataset`.

    Returns:
        A tuple containing (mean, std) for the entire dataset.
    """

    # Extract images and labels
    data_r = np.stack([np.array(dataset[i][0])[:, :, 0] for i in range(len(dataset))])
    data_g = np.stack([np.array(dataset[i][0])[:, :, 1] for i in range(len(dataset))])
    data_b = np.stack([np.array(dataset[i][0])[:, :, 2] for i in range(len(dataset))])

    # Compute mean and std
    mean = np.mean(data_r), np.mean(data_g), np.mean(data_b)
    std = np.std(data_r), np.std(data_g), np.std(data_b)

    return mean, std


# Define LeNet-5 architecture
class LeNet5(nn.Module):
    def __init__(self):
        super(LeNet5, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, 5)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(64, 64, 5)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 5 * 5, 384)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(384, 192)
        self.relu4 = nn.ReLU()
        self.fc3 = nn.Linear(192, 100)

    def forward(self, x):
        x = self.pool1(self.relu1(self.conv1(x)))
        x = self.pool2(self.relu2(self.conv2(x)))
        x = x.view(-1, 64 * 5 * 5)
        x = self.relu3(self.fc1(x))
        x = self.relu4(self.fc2(x))
        x = self.fc3(x)
        return x

# Command-line arguments
parser = argparse.ArgumentParser(description='Train with Local SGD')
parser.add_argument('--k', type=int, default=2, help='choose a K value for local SGD: [2, 4, 8]')
parser.add_argument('--j', type=int, default=4, help='choose a J value for local SGD: [4, 8, 16, 32, 64]')
args = parser.parse_args()
K = args.k
J = args.j
print(f"Training with K={K}, J={J}")

global_iteration_per_epoch = math.ceil(global_iteration_per_epoch / (K * J)) * (K * J)
global_iteration_per_epoch = global_iteration_per_epoch // (K * J)
print(f"Global iteration per epoch: {global_iteration_per_epoch}")


# Device setup
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

torch.manual_seed(42) # Set the seed for reproducibility
torch.cuda.manual_seed_all(42) # Set the seed for reproducibility on GPU

# use the same mean and std to add consistency to all datasets
data = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transforms.ToTensor())
mean, std = compute_mean_std(data)

# Load and split CIFAR-100 dataset
train_transform = transforms.Compose([
  transforms.RandomCrop(32, padding=4),
  transforms.RandomHorizontalFlip(),
  transforms.ToTensor(),
  transforms.Normalize(mean, std)
])

test_transform = transforms.Compose([
    transforms.ToTensor(),  # Convert to PyTorch tensor
    transforms.Normalize(mean, std)
])

trainset = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=train_transform)

# Shard the training set
shard_size = len(trainset) // K
shards = [Subset(trainset, range(i * shard_size, (i + 1) * shard_size)) for i in range(K)]
# After creating the DataLoader for each worker, create an iterator
shard_iterators = [iter(DataLoader(shards[k], batch_size=batch_size, shuffle=True)) for k in range(K)]


# Test set
testset = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=test_transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

# Scale the learning rate by k
learning_rate = base_learning_rate * K
print(f"Base learning rate for each k: {learning_rate:.5f}")
#lr_warmup=learning_rate
# Initialize local models, optimizers and scheduler
nets = [LeNet5().to(device) for _ in range(K)]


with torch.no_grad():
    for net in nets[1:]:  # Skip nets[0] because it's the reference model
        for param_target, param_source in zip(net.parameters(), nets[0].parameters()):
            param_target.data.copy_(param_source.data)

optimizers = [optim.SGD(net.parameters(), lr=learning_rate, momentum=momentum, weight_decay=weight_decay) for net in nets]
schedulers = [optim.lr_scheduler.CosineAnnealingLR(optimizer=optimizer, T_max=num_epochs) for optimizer in optimizers]

u_buffer = [torch.zeros_like(param) for param in nets[0].parameters()]  # Momentum buffer

criterion = nn.CrossEntropyLoss()

# Initialize storage for train losses and accuracies for each worker
train_losses_per_worker = [[] for _ in range(K)]
train_accuracies_per_worker = [[] for _ in range(K)]

# Initialize test losses and accuracies for test set
test_losses = []
test_accuracies = []

# Training loop
for epoch in range(num_epochs):
    # print(f"Epoch {epoch + 1}/{num_epochs}:")
    #if epoch < warmup_epochs: #lr for buffer update during warmup
            #w_factor = (epoch + 1) / warmup_epochs
            #lr_warmup = w_factor * learning_rate
    train_loss_for_iteration = [0.0 for _ in range(K)]
    correct_train_for_iteration = [0 for _ in range(K)]
    total_train_for_iteration = [0 for _ in range(K)]
    for iteration in range(global_iteration_per_epoch): #outer loop

        #x(t,0)
        iter_params = [torch.zeros_like(param) for param in nets[0].parameters()]
        for param, g_param in zip(iter_params, nets[0].parameters()):
                    param.data.copy_(g_param)
        # Sequentially train each worker to simulate a distributed environment
        for k in range(K):

            nets[k].train()
            correct_train, total_train, train_loss = 0, 0, 0.0

            # Perform J local step before synchronization
            for j in range(J): #inner loop
                      try:
                          # Get a batch from the current worker's shard using next()
                          images, labels = next(shard_iterators[k])
                          images, labels = images.to(device), labels.to(device)  # Move to device
                          optimizers[k].zero_grad()
                          outputs = nets[k](images)
                          loss = criterion(outputs, labels)
                          loss.backward()
                          optimizers[k].step()
                          # Calculate train loss and accuracy
                          train_loss += loss.item() * labels.size(0)
                          _, predicted = outputs.max(1)
                          total_train += labels.size(0)
                          correct_train += predicted.eq(labels).sum().item()

                      except StopIteration:
                          # If the iterator has exhausted the data, restart the iterator
                          shard_iterators[k] = iter(DataLoader(shards[k], batch_size=batch_size, shuffle=True))
                          images, labels = next(shard_iterators[k]) # Resume from the next batch
                          images, labels = images.to(device), labels.to(device)
                          # Perform optimization step
                          optimizers[k].zero_grad()
                          outputs = nets[k](images)
                          loss = criterion(outputs, labels)
                          loss.backward()
                          optimizers[k].step()
                          # Calculate train loss and accuracy
                          train_loss += loss.item() * labels.size(0)
                          _, predicted = outputs.max(1)
                          total_train += labels.size(0)
                          correct_train += predicted.eq(labels).sum().item()

            train_loss_for_iteration[k] += train_loss
            correct_train_for_iteration[k] += correct_train
            total_train_for_iteration[k] += total_train


         # Synchronization (SLOWMO Algothm 1 - paper 21)
        with torch.no_grad():
            global_params = [torch.zeros_like(param) for param in nets[0].parameters()]
            #x(t, tau)
            for net in nets:
                for global_param, local_param in zip(global_params, net.parameters()):
                    global_param += local_param
            for global_param in global_params:
                global_param /= K

            for param, g_param, u_param in zip(iter_params, global_params, u_buffer):
                #if epoch < warmup_epochs:
                  #  u_param.mul_(beta).add_((param - g_param) / lr_warmup)
                 #   g_param.sub_(alpha * lr_warmup * u_param)

                #else: #Sync after warmup
                    #print(f"{optimizers[0].param_groups[0]['lr']}  {optimizers[1].param_groups[0]['lr']}")
                    lr_sched = optimizers[0].param_groups[0]['lr'] #optimizer lr
                    u_param.mul_(beta).add_((param - g_param) / lr_sched)
                    #g_param.sub_(alpha * lr_sched * u_param)
                    g_param.copy_(param - alpha * lr_sched * u_param)

            for net in nets:
                for param, g_param in zip(net.parameters(), global_params):
                    param.data.copy_(g_param)

    # Update lr for each scheduler
    for scheduler in schedulers:
        #if epoch >= warmup_epochs:
            scheduler.step()  # Trigger cosine decay
        #else:
         #   warmup_factor = (epoch + 1) / warmup_epochs
          #  lr = warmup_factor * learning_rate
           # for optimizer in optimizers:
            #    for param_group in optimizer.param_groups:
             #       param_group['lr'] = lr
        # print(f"Learning rate at epoch {epoch + 1}: {scheduler.get_lr()[0]:.5f}")

    # Final Evaluation on Test Set
    global_model = nets[0] # all models are synchronized
    global_model.eval()
    # Test
    correct_test, total_test, test_loss = 0, 0, 0.0
    with torch.no_grad():
        for inputs, labels in testloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = global_model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total_test += labels.size(0)
            correct_test += predicted.eq(labels).sum().item()
    test_loss /= len(testloader)
    test_losses.append(test_loss)
    test_accuracy = 100. * correct_test / total_test
    test_accuracies.append(test_accuracy)

    print(f"Epoch {epoch + 1}/{num_epochs}: Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%")
    for k in range(K):
        # Store train loss and accuracy for this worker
        train_loss_for_iteration[k] /= total_train_for_iteration[k]
        train_losses_per_worker[k].append(train_loss_for_iteration[k])
        tot_accuracy = 100. * correct_train_for_iteration[k] / total_train_for_iteration[k]
        train_accuracies_per_worker[k].append(tot_accuracy)
        print(f"    Worker {k}: Train Loss: {train_losses_per_worker[k][-1]:.4f}, Train Accuracy: {train_accuracies_per_worker[k][-1]:.2f}%")

# Plot results
# Plot Train Loss for Each Worker
plt.figure(figsize=(8, 6))
for k in range(K):
    plt.plot(train_losses_per_worker[k], label=f'Worker {k} Train Loss')
plt.title('Train Loss per Worker')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.tight_layout()
plt.savefig('train_loss_per_worker.png')
plt.show()

# Plot Train Accuracy for Each Worker
plt.figure(figsize=(8, 6))
for k in range(K):
    plt.plot(train_accuracies_per_worker[k], label=f'Worker {k} Train Accuracy')
plt.title('Train Accuracy per Worker')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.legend()
plt.tight_layout()
plt.savefig('train_accuracy_per_worker.png')
plt.show()

# Plot Test Loss
plt.figure(figsize=(6, 4))
plt.plot(test_losses, label='Test Loss')
plt.title('Test Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.tight_layout()
plt.savefig('test_loss.png')
plt.show()

# Plot Test Accuracy
plt.figure(figsize=(6, 4))
plt.plot(test_accuracies, label='Test Accuracy')
plt.title('Test Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.legend()
plt.tight_layout()
plt.savefig('test_accuracy.png')
plt.show()

Overwriting localSGD.py


In [6]:
%run localSGD.py --k 3 --j 8

Training with K=3, J=8
Global iteration per epoch: 33
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Base learning rate for each k: 0.00300
Epoch 1/150: Test Loss: 4.3509, Test Accuracy: 4.09%
    Worker 0: Train Loss: 4.5585, Train Accuracy: 2.00%
    Worker 1: Train Loss: 4.5557, Train Accuracy: 2.10%
    Worker 2: Train Loss: 4.5603, Train Accuracy: 2.03%
Epoch 2/150: Test Loss: 3.9396, Test Accuracy: 9.46%
    Worker 0: Train Loss: 4.1320, Train Accuracy: 6.93%
    Worker 1: Train Loss: 4.1212, Train Accuracy: 7.20%
    Worker 2: Train Loss: 4.1312, Train Accuracy: 6.85%
Epoch 3/150: Test Loss: 3.7427, Test Accuracy: 12.92%
    Worker 0: Train Loss: 3.8813, Train Accuracy: 10.38%
    Worker 1: Train Loss: 3.8568, Train Accuracy: 11.08%
    Worker 2: Train Loss: 3.8705, Train Accuracy: 10.42%
Epoch 4/150: Test Loss: 3.5505, Test Accuracy: 15.96%
    Worker 0: Train Loss: 3.6956, Train Accuracy: 13.04%
    Worker 1: T

KeyboardInterrupt: 